# Cleaning Sequences
There are multiple ways of sequencing the steps in your data cleaning process. We've used one so far, once in Lesson 1 and once in the **Data Cleaning Process** example video in this lesson. The **Define**, **Code**, and **Test** Markdown headers were used once in this sequence, with multiple definitions, cleaning operations, and tests under each header, respectively.

It looked like this:

______

## Gather

In [1]:
import pandas as pd
import requests
import tweepy
import timeit

#### Keys

In [2]:
consumer_key = 'wp5lVM9qitFOplu1dWeQ34x26' 
consumer_secret = 'iXPxPQ0HEO14U8wMwwiLugMwZbpeadPWRbGSjNFI3jHriwQmKA'

access_token = '739672993-sESdNtfAq523OsJduxGBUmUnZLSfz4274y7E3eCn' 
access_token_secret = 'vqYxHjS2sEPRfRHjdEv145h5WbtgtVQiTh46krp3ZB4FC' 

In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
req = requests.get(url, auth=('user','pass'));
open('image_predictions.tsv', 'wb').write(req.content);

KeyboardInterrupt: 

In [ ]:
img_pred = pd.read_csv('image_predictions.tsv', sep='\t');

In [ ]:
img_pred.head()

In [ ]:
#Query Twitter API for Tweet JSON data
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#Call API
api = tweepy.API(auth)


#api.get_status(tweet_id, tweet_mode='extended')

In [ ]:
num_tweets = 200
start = timeit.timeit()
tweets = api.user_timeline(screen_name=WeRateDogs, count = num_tweets)
end = timeit.timeit()
tweets_for_csv = [tweet.text for tweet in tweets]
print("Gathering Tweets took a total of ", end - start)

## Assess

#### Quality
- 

## Clean

#### Define
- Remove 'bb' before every animal name using string slicing
- Replace ! with . in body weight and brain weight columns

#### Code

#### Test

________

But you can also use multiple **Define**, **Code**, and **Test** headers, one for each data quality and tidiness issue (or group of data quality and tidiness issues). Effectively, you are defining then coding then testing immediately. This sequence is helpful when you have a lot of quality and tidiness issues to clean. Since that is the case in this lesson, this sequence will be used.

Pasting each assessment above the **Define** header as its own header can also be helpful.

Here's what this sequence looks like using the *animals.csv* dataset (and reusing the above *Gather* and *Assess* steps):

_______

## Clean

#### bb before every animal name
##### Define
Remove 'bb' before every animal name using string slicing.

##### Code

##### Test

#### ! instead of . for decimal in body weight and brain weight
##### Define
Replace ! with . in body weight and brain weight columns

##### Code

##### Test